# Amit Kumar
# 22CSM1R02
# ML - Assignment 4 - Genetic Algorithms

![](./problem.png)

- population generation
- encoding technique (gene, chromosome)
- initialization procedure (creation)
- evaluation/fitness function (environment)
- selection of parents (reproduction)
- genetic operators (mutation/recombination)
- parameter setting

In [1]:
import numpy as np
import math
import random

In [2]:
'''
Class for Genetic Algorithm that optimized a function with two independent variables, x and y
Constructor Params
    - initial values of paramenters, x and y
'''
class GenericAlgorithm:
    def __init__(self, size, lower_lim=None, upper_lim=None):
        self.n = size # size of population
        self.l = lower_lim
        self.u = upper_lim
        
        
    def fit(self, n_iters=5):
        pops = self.population_generation() # initial population
        fitness = []
        for i in range(n_iters):
            print('Iteration: ', i)
            fitness = self.find_fitness(pops)
            
            # for debugging
            # for j in range(len(pops)):
            #     x1,y1=convert_to_int(pops[j])
            #     t1 = (x1+(2*y1)-2)**2
            #     t2 = ((2*x1)+y1-7)**2
            #     print(f"x {x1} y {y1} sum {t1+t2} fit {fitness[j]}")
                
            print('Total Fitness of Population ', i, ': ', end='')
            
            print(self.fitness_score(fitness))
            print()
            # generating the next population, crossover and then mutations and then next iterations
            pops = self.next_gen(pops, fitness)
            pops = self.mutation(pops)
        return (pops, self.fitness_score(fitness))
        
        
    def population_generation(self):
        '''
        Generate population randomly using bit strings of length 10bits
        Range of number is [-10,10], 10 fits in 4 bits, so we take length 5 bit strings
            - first char is - then it is negative 
            - if it is zero then it is positive
        Data Structure for storing the population
            - The population is stored in a list and each element of the list is a 3 element dictionary
                - element 1, combined bit string of length 8 -> pop
                - element 2, sign of first part -> sign1
                - element 3, sign of second part -> sign2
                
        Inputs
        Outputs
        '''
        population = []
        for i in range(self.n):
            population.append(self._generate_pop())
        return population
        
        
    def _generate_pop(self):
        x = random.randint(-10, 10)
        y = random.randint(-10, 10)
        x_s = f'{x:05b}'
        y_s = f'{y:05b}'
        sign1 = x_s[0]
        sign2 = y_s[0]
        x_s = x_s[1:]
        y_s = y_s[1:]
        return {
            'pop': x_s+y_s,
            'sign1': sign1, 
            'sign2': sign2
        }
    
    
    def find_fitness(self, population):
        fitness = []
        for pop in population:
            fitness.append(self._fitness_function(pop))
        return fitness
    
    
    def _fitness_function(self, pop):
        '''
        Takes an individual pop and finds the value of fitness function for this pop
        '''
        x, y = self._convert_to_int(pop)
        t1 = (x+2*y-2)**2
        t2 = (2*x+y-7)**2
        if (t1+t2==0):
            return 1
        return 1/abs(t1+t2)
        
        
    def _convert_to_int(self, pop):
        p1 = pop['pop'][:4]
        p2 = pop['pop'][4:]
        v1 = 0
        v2 = 0
        if pop['sign1'] == '0': 
            v1 = int(p1, 2)
        else:
            v1 = -1*int(p1, 2)
            
        if pop['sign2'] == '0': 
            v2 = int(p2, 2)
        else:
            v2 = -1*int(p2, 2) 
            
        return (v1, v2)
    
    
    def fitness_score(self, fitness):
        return sum(fitness)
    
    
    def next_gen(self, population, fitness): 
        new_population = self._sort_population_on_fitness(population, fitness)
        #replacing least 2 results with highest result
        new_population[0]=new_population[-1].copy()
        new_population[1]=new_population[-2].copy()
        # random crossover taking two pops ar a time
        
        for i in range(0, self.n, 2):
            x1, x2 = self._cross_over(new_population[i], new_population[i+1])
            new_population[i] = x1
            new_population[i+1] = x2
        return new_population
            
            
    def _cross_over(self, pop1, pop2):
        # 1 point crossover,
        k = random.randint(1, 7)
        s1 = list(pop1['pop'])
        s2 = list(pop2['pop'])
        temp = s1[k:]
        s1[k:] = s2[k:]
        s2[k:] = temp
        s1 = ''.join(s1)
        s2 = ''.join(s2)
        pop1['pop'] = s1
        pop2['pop'] = s2
        return (pop1, pop2)
        
        
    def _sort_population_on_fitness(self, population, fitness):
        return [x for _, x in sorted(zip(fitness, population), key = lambda x: x[0])]
    
    
    def mutation(self, population):
        '''
        Mutation applied by flipping k bits, k selected randomly
        '''
        k = random.randint(1, 7)
        for i in range(len(population)):
            population[i] = self._flip_bits(population[i], k)
        return population
            
            
    def _flip_bits(self, pop, k):
        rand_perm = list(np.random.permutation(8)) # because 8 bit length strings
        # take the first k every time and flip them
        s = list(pop['pop'])
        for i in range(k):
            if s[rand_perm[i]] == '0':
                s[rand_perm[i]] = '1'
            else:
                s[rand_perm[i]] = '0'
        pop['pop'] = ''.join(s)
        return pop

# Running the Algorithm

In [3]:
def convert_to_int(pop):
    p1 = pop['pop'][:4]
    p2 = pop['pop'][4:]
    v1 = 0
    v2 = 0
    if pop['sign1'] == '0': 
        v1 = int(p1, 2)
    else:
        v1 = -1*int(p1, 2)

    if pop['sign2'] == '0': 
        v2 = int(p2, 2)
    else:
        v2 = -1*int(p2, 2) 

    return (v1, v2)

In [4]:
ga = GenericAlgorithm(size=10)
n_iters = 100
output = ga.fit(n_iters)
print('Fitness after {} iterations {}'.format(n_iters ,output[1]))

min=9999999
minx=0
miny=0
for i in output[0]:
    x1,y1=convert_to_int(i)
    t1 = (x1+(2*y1)-2)**2
    t2 = ((2*x1)+y1-7)**2
    if(min>(t1+t2)):
        min=t1+t2
        minx=x1
        miny=y1
        
print(f"Most optimized result: x={minx} y={miny} f(x,y)={min}")

Iteration:  0
Total Fitness of Population  0 : 0.15012039360326146

Iteration:  1
Total Fitness of Population  1 : 0.09131116065857464

Iteration:  2
Total Fitness of Population  2 : 0.5723720710706652

Iteration:  3
Total Fitness of Population  3 : 0.046426189132754786

Iteration:  4
Total Fitness of Population  4 : 0.04298277256009664

Iteration:  5
Total Fitness of Population  5 : 0.26279283039475737

Iteration:  6
Total Fitness of Population  6 : 0.27255859025878104

Iteration:  7
Total Fitness of Population  7 : 0.06143388111245272

Iteration:  8
Total Fitness of Population  8 : 0.25332158165213886

Iteration:  9
Total Fitness of Population  9 : 0.09190081012106947

Iteration:  10
Total Fitness of Population  10 : 0.09863287868277248

Iteration:  11
Total Fitness of Population  11 : 0.09973378068211394

Iteration:  12
Total Fitness of Population  12 : 0.050486846458350346

Iteration:  13
Total Fitness of Population  13 : 0.09288683285479182

Iteration:  14
Total Fitness of Populat

# Rough Work

In [4]:
x = -10
y = 10
binary1 = f'{x:05b}'
binary2 = f'{y:05b}'
print(binary1, binary2)
print(binary1[0], binary2[0])
print(binary1[1:], binary2[1:])
binary1 = binary1[1:]
binary2 = binary2[1:]
final=binary1+binary2
print(final)

-1010 01010
- 0
1010 1010
10101010


In [5]:
def _generate_pop():
        x = random.randint(-10, 10)
        y = random.randint(-10, 10)
        x_s = f'{x:05b}'
        y_s = f'{y:05b}'
        sign1 = x_s[0]
        sign2 = y_s[0]
        x_s = x_s[1:]
        y_s = y_s[1:]
        return {
            'pop': x_s+y_s,
            'sign1': sign1, 
            'sign2': sign2
        }

In [6]:
print(_generate_pop())

{'pop': '10010010', 'sign1': '0', 'sign2': '-'}


In [7]:
p1 = final[:4]
p2 = final[4:]
print(final)
print(p1, p2)
v1 = int(p1, 2)
v2 = int(p2, 2)
print(v1, v2)

10101010
1010 1010
10 10


In [8]:
def _convert_to_int(pop):
        p1 = pop['pop'][:4]
        p2 = pop['pop'][4:]
        v1 = 0
        v2 = 0
        if pop['sign1'] == '0': 
            v1 = int(p1, 2)
        else:
            v1 = -1*int(p1, 2)
            
        if pop['sign2'] == '0': 
            v2 = int(p2, 2)
        else:
            v2 = -1*int(p2, 2) 
            
        return (v1, v2)

In [9]:
print(_convert_to_int(_generate_pop()))

(-9, -6)


In [10]:
(2+3)**2

25

In [11]:
def _fitness_function(pop):
        '''
        Takes an individual pop and finds the value of fitness function for this pop
        '''
        x, y = _convert_to_int(pop)
        t1 = (x+2*y-2)**2
        t2 = (2*x+y-7)**2
        return 1/(t1+t2)

In [12]:
p = _generate_pop()
print(_fitness_function(p))

0.006172839506172839


In [13]:
sum([1, 2, 3, 4, 5])

15

In [14]:
X = ['Amit', 'Naman', 'Bhautik', 'Anmol', 'Rushikesh', 'Darshan', 'Iswar', 'Dishant']
fitness = [1, 6, 2, 8, 5, 7, 3, 4]
print(X)
print(fitness)
X = [x for _, x in sorted(zip(fitness, X))]
print(X)

['Amit', 'Naman', 'Bhautik', 'Anmol', 'Rushikesh', 'Darshan', 'Iswar', 'Dishant']
[1, 6, 2, 8, 5, 7, 3, 4]
['Amit', 'Bhautik', 'Iswar', 'Dishant', 'Rushikesh', 'Naman', 'Darshan', 'Anmol']


In [15]:
3&1

1

In [16]:
2&1

0

In [17]:
t = -100
print(f'{t:b}')

-1100100


In [18]:
k = random.randint(1, 8)
k

5

In [19]:
s1 = '01110000'
s1 = list(s1)
s2 = '10101111'
s2 = list(s2)
k = 7
temp = s1[k:]
s1[k:] = s2[k:]
s2[k:] = temp
s1 = ''.join(s1)
s2 = ''.join(s2)
print(s1)
print(s2)

01110001
10101110


In [20]:
# bit flipping
x = list(np.random.permutation(10))
x

[0, 5, 9, 6, 4, 7, 3, 2, 1, 8]

In [21]:
def flip_bits(pop, k):
    rand_perm = list(np.random.permutation(8)) # because 8 bit length strings
    print(rand_perm)
    # take the first k every time and flip them
    s = list(pop['pop'])
    print(s)
    for i in range(k):
        if s[rand_perm[i]] == '0':
            s[rand_perm[i]] = '1'
        else:
            s[rand_perm[i]] = '0'
    pop['pop'] = ''.join(s)
    print(s)
    return pop['pop']

In [22]:
p = _generate_pop()
print(p)

{'pop': '10100100', 'sign1': '-', 'sign2': '0'}


In [23]:
flip_bits(p, 3)

[7, 2, 4, 5, 1, 3, 6, 0]
['1', '0', '1', '0', '0', '1', '0', '0']
['1', '0', '0', '0', '1', '1', '0', '1']


'10001101'